this code have been debug in pycharm, and present in jupyter and colab for execute in cloud

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    pass

# 修改当前文件夹位置 假定notebook文件就在项目文件夹根目录
import os
def get_root_dir():
    pathname = '/mcapplus/' # the name of sync folder
    if os.path.exists('/content/drive/MyDrive'+pathname):
        return '/content/drive/MyDrive/' + pathname #在Colab里
    else:
        return './' #在本地

#调用系统命令，相当于cd，但是直接!cd是不行的
os.chdir(get_root_dir())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
path_sync = os.getcwd()

In [3]:
'''
尝试实现biaco
    结果记录
大系统实验思路
    对于每个系统，生成10000个合法解（此时不知道哪个解好哪个解不好），然后存储起来，等到确定type和seed之后才能计算知道最优和最差解

'''
#%% 导入包
import numpy as np
import pandas as pd
from codehub_mcapplus import Common,Project,Case
import math
import copy
import time
import os
# import heartrate
# heartrate.trace(browser=True)
#%% small system
pattern = ['g','f']
nk = [[2,7],[3,7],[2,8],[3,8],[3,9],[4,9],[3,10],[4,10]]
# ,[3,12],[4,12],[3,15],[4,15] 小系统不需要
condition = ['less']
namelist_small = []

for n in nk:
    for p in pattern:
        for c in condition:
            namelist_small.append(p + '_' + str(n[0]) + '_' + str(n[1]) + '_' + c)
print(namelist_small)
# path
path_small = path_sync + '//result//now//smallsystem//'
#%% largesystem
pattern = ['g','f']
nk = [[4,20],[5,20],[10,20],[12,30],[15,30],[20,30],[22,40],[25,40],[30,40],[32,50],[35,50],[40,50]]
condition = ['less']
namelist_large = []
for p in pattern:
    for n in nk:
        for c in condition:
            namelist_large.append(p + '_' + str(n[0]) + '_' + str(n[1]) + '_' + c)
print(namelist_large)
# path
path_large = '.\\result\\now\\largesystem\\'

['g_2_7_less', 'f_2_7_less', 'g_3_7_less', 'f_3_7_less', 'g_2_8_less', 'f_2_8_less', 'g_3_8_less', 'f_3_8_less', 'g_3_9_less', 'f_3_9_less', 'g_4_9_less', 'f_4_9_less', 'g_3_10_less', 'f_3_10_less', 'g_4_10_less', 'f_4_10_less']
['g_4_20_less', 'g_5_20_less', 'g_10_20_less', 'g_12_30_less', 'g_15_30_less', 'g_20_30_less', 'g_22_40_less', 'g_25_40_less', 'g_30_40_less', 'g_32_50_less', 'g_35_50_less', 'g_40_50_less', 'f_4_20_less', 'f_5_20_less', 'f_10_20_less', 'f_12_30_less', 'f_15_30_less', 'f_20_30_less', 'f_22_40_less', 'f_25_40_less', 'f_30_40_less', 'f_32_50_less', 'f_35_50_less', 'f_40_50_less']


In [4]:
#%% 记录小系统结果
def saveDataFrame(df,path,savetype):
    if savetype == 'cover':
        df.to_csv(path,header=None)
    elif os.path.exists(path) != True:#不覆盖且不存在文件
        df.to_csv(path,header=None)

def smallsystem(namelist):
    ssr_list = []
    Seednum = 1
    bias = 2
    seed_range = np.arange(bias,Seednum+bias)
    for name_index in range(0,len(namelist)):
        name = namelist[name_index]
        df = pd.DataFrame(index=seed_range,
                          columns= pd.MultiIndex.from_product([['low','high','arbitrary'],['ssr','sys','design','max_sys','best_design','min_sys','worst_design','processtime','time','enumerationprotime','enumerationtime']])
                          )
        for type in range(1, 3 + 1):
            typename_dict = {1:'low',2:'high',3:'arbitrary'}
            for seed in seed_range:
                common_obj = Common()
                case_obj = Case()
                k,pattern,plist,region,positionnum,pnums,problem_type = case_obj.getcase(name,type,seed)
                region_dict,position_dict,position_type_dict,position_index_dict = common_obj.get_regiondict(region_2D=region)
                component_type_dict,component_index_dict = common_obj.get_componentdict(pnums)
                project_obj = Project(region_dict,position_dict,position_type_dict,position_index_dict,component_index_dict,component_type_dict,plist,pattern,k,pnums,seed)
                #%%
                start_processtime = time.process_time()
                start_time = time.time()
                design,sys,iter = project_obj.BIACO()
                end_processtime = time.process_time()
                end_time = time.time()
                biaco_time = end_time - start_time
                biaco_processtime = end_processtime - start_processtime
                # print('sys:{}'.format(sys))
                #%% test enumeration
                start_enumerationprotime = time.process_time()
                start_enumerationtime = time.time()
                (best_design,best_sys),(worst_design,worst_sys) = project_obj.enumeration()
                end_enumerationprotime = time.process_time()
                end_enumerationtime = time.time()
                enumeration_time = end_enumerationtime - start_enumerationtime
                enumeration_protime = end_enumerationprotime - start_enumerationprotime
                # print('best design:{},best sys:{}'.format(best_design,best_sys))
                # print('worst design:{},worst sys:{}'.format(worst_design,worst_sys))
                #%%
                SSR = (sys - worst_sys) / (best_sys - worst_sys)
                # print('SSR:{}'.format(SSR))
                ssr_list.append(SSR)
                # 记录
                typename = typename_dict[type]
                record = np.array([SSR,sys,design,best_sys,best_design,worst_sys,worst_design,biaco_processtime,biaco_time,enumeration_protime,enumeration_time])
                df.loc[seed,(typename,slice(None))] = record
        filepath = path_small + name + '_seed_'+ str(Seednum) + '.csv'
        saveDataFrame(df,filepath,savetype='nocover')
        print('name:{},SSR:{}'.format(name,np.mean(ssr_list)))
smallsystem(namelist_small)

OSError: ignored

In [ ]:
    # 汇总内容,做一个函数
def largesystem(namelist):
    ssr_list = []
    Seednum = 1
    seed_range = np.arange(Seednum)
    for name_index in range(0,len(namelist)):
        name = namelist[name_index]
        df = pd.DataFrame(index=seed_range,
                          columns= pd.MultiIndex.from_product([['low','high','arbitrary'],['ssr','sys','design','max_sys','best_design','min_sys','worst_design','processtime','time','randomprotime','randomtime']])
                          )
        common_obj = Common()
        case_obj = Case()
        k_r, pattern_r, plist_r, region_r, positionnum_r, pnums_r, problem_type = case_obj.getcase(name, 1, 0)
        region_dict_r, position_dict_r, position_type_dict_r, position_index_dict_r = common_obj.get_regiondict(
            region_2D=region_r)
        component_type_dict_r, component_index_dict_r = common_obj.get_componentdict(pnums_r)
        project_obj_r = Project(region_dict_r, position_dict_r, position_type_dict_r, position_index_dict_r, component_index_dict_r,
                              component_type_dict_r, plist_r, pattern_r, k_r, pnums_r, 0)
        print('yes1')
        design_pop, method_time = project_obj_r.random_method()
        print('yes2')
        for type in range(1, 3 + 1):
            typename_dict = {1:'low',2:'high',3:'arbitrary'}
            for seed in range(len(seed_range)):
                common_obj = Common()
                case_obj = Case()
                k,pattern,plist,region,positionnum,pnums,problem_type = case_obj.getcase(name,type,seed)
                region_dict,position_dict,position_type_dict,position_index_dict = common_obj.get_regiondict(region_2D=region)
                component_type_dict,component_index_dict = common_obj.get_componentdict(pnums)
                project_obj = Project(region_dict,position_dict,position_type_dict,position_index_dict,component_index_dict,component_type_dict,plist,pattern,k,pnums,seed)
                #%%
                print('yes')
                start_processtime = time.process_time()
                start_time = time.time()
                design,sys,iter = project_obj.BIACO()
                end_processtime = time.process_time()
                end_time = time.time()
                biaco_time = end_time - start_time
                biaco_processtime = end_processtime - start_processtime
                # print('sys:{}'.format(sys))
                #%% test enumeration
                start_enumerationprotime = time.process_time()
                start_enumerationtime = time.time()
                random_sys = np.array([project_obj.system_reliability(project_obj.transfer_design(i)) for i in design_pop])
                best_sys = np.max(random_sys)
                best_design = design_pop[np.argmax(random_sys)]
                worst_sys = np.min(random_sys)
                worst_design = design_pop[np.argmin(random_sys)]
                end_enumerationprotime = time.process_time()
                end_enumerationtime = time.time()
                enumeration_time = end_enumerationtime - start_enumerationtime + method_time / Seednum
                enumeration_protime = end_enumerationprotime - start_enumerationprotime + method_time / Seednum
                # print('best design:{},best sys:{}'.format(best_design,best_sys))
                # print('worst design:{},worst sys:{}'.format(worst_design,worst_sys))
                #%%
                SSR = (sys - worst_sys) / (best_sys - worst_sys)
                # print('SSR:{}'.format(SSR))
                ssr_list.append(SSR)
                # 记录
                typename = typename_dict[type]
                record = np.array([SSR,sys,design,best_sys,best_design,worst_sys,worst_design,biaco_processtime,biaco_time,enumeration_protime,enumeration_time])
                df.loc[seed,(typename,slice(None))] = record
        filepath = path_small + name + '_seed_'+ str(Seednum) + '.csv'
        saveDataFrame(df,filepath,savetype='cover')
        print('name:{},SSR:{}'.format(name,np.mean(ssr_list)))


#%%


# largesystem(namelist_large)

In [7]:
os.path.isfile('.\\Win32Project2.dll')

False